# Podjetje, varianta 1

In [1]:
from sqlalchemy import Column, DateTime, String, Integer, ForeignKey, func, select, create_engine
from sqlalchemy.orm import declarative_base, relationship, backref, sessionmaker
from datetime import datetime

In [2]:
Base = declarative_base()


class Oddelek(Base):
    __tablename__ = 'oddelek'

    id = Column(Integer, primary_key=True)
    naziv = Column(String)

    def __repr__(self):
        return f"Oddelek[{self.id}, {self.naziv}, {self.seznamZaposlenih}]"


class Zaposleni(Base):
    __tablename__ = 'zaposleni'

    id = Column(Integer, primary_key=True)
    ime = Column(String)

    # uporabimo razred func za dostop do funkcij na bazi, npr. func.now()
    datum_zaposlitve = Column(DateTime, default=func.now())
    oddelek_id = Column(Integer, ForeignKey('oddelek.id'))

    # cascade='delete,all' bo povzročil brisanje vseh zaposlenih v oddelku
    oddelek = relationship(Oddelek,
                           backref=backref('seznamZaposlenih', uselist=True, cascade='delete,all'))

    def __repr__(self):
        return f"Zaposleni[{self.id}, {self.ime}, {self.datum_zaposlitve}, {self.oddelek_id}]"

In [3]:
engine = create_engine('sqlite:///podjetje1.db', echo=True)
Base.metadata.bind = engine
Base.metadata.create_all(engine)

2025-06-05 10:34:50,880 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-06-05 10:34:50,881 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("oddelek")
2025-06-05 10:34:50,883 INFO sqlalchemy.engine.Engine [raw sql] ()
2025-06-05 10:34:50,884 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("oddelek")
2025-06-05 10:34:50,885 INFO sqlalchemy.engine.Engine [raw sql] ()
2025-06-05 10:34:50,886 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("zaposleni")
2025-06-05 10:34:50,886 INFO sqlalchemy.engine.Engine [raw sql] ()
2025-06-05 10:34:50,887 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("zaposleni")
2025-06-05 10:34:50,888 INFO sqlalchemy.engine.Engine [raw sql] ()
2025-06-05 10:34:50,889 INFO sqlalchemy.engine.Engine 
CREATE TABLE oddelek (
	id INTEGER NOT NULL, 
	naziv VARCHAR, 
	PRIMARY KEY (id)
)


2025-06-05 10:34:50,890 INFO sqlalchemy.engine.Engine [no key 0.00068s] ()
2025-06-05 10:34:50,900 INFO sqlalchemy.engine.Engine 
CREATE TABLE zaposleni (
	id INTEGE

In [4]:
DBSessionMaker = sessionmaker(bind=engine)
session = DBSessionMaker()

Ustvarimo oddelek.

In [5]:
(oddelek1 := Oddelek(naziv="IT"))

Oddelek[None, IT, []]

Ustvarimo zaposlenega.

In [6]:
(zaposleni1 := Zaposleni(ime="Janez", oddelek=oddelek1))

Zaposleni[None, Janez, None, None]

Dodajmo ustvarjena objekta v bazo.

In [7]:
session.add(oddelek1)
session.add(zaposleni1)
session.commit()

2025-06-05 10:35:53,736 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-06-05 10:35:53,739 INFO sqlalchemy.engine.Engine INSERT INTO oddelek (naziv) VALUES (?)
2025-06-05 10:35:53,739 INFO sqlalchemy.engine.Engine [generated in 0.00066s] ('IT',)
2025-06-05 10:35:53,742 INFO sqlalchemy.engine.Engine INSERT INTO zaposleni (ime, datum_zaposlitve, oddelek_id) VALUES (?, CURRENT_TIMESTAMP, ?)
2025-06-05 10:35:53,742 INFO sqlalchemy.engine.Engine [generated in 0.00045s] ('Janez', 1)
2025-06-05 10:35:53,743 INFO sqlalchemy.engine.Engine COMMIT


In [8]:
oddelek1, zaposleni1

2025-06-05 10:36:55,408 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-06-05 10:36:55,411 INFO sqlalchemy.engine.Engine SELECT oddelek.id AS oddelek_id, oddelek.naziv AS oddelek_naziv 
FROM oddelek 
WHERE oddelek.id = ?
2025-06-05 10:36:55,412 INFO sqlalchemy.engine.Engine [generated in 0.00074s] (1,)
2025-06-05 10:36:55,415 INFO sqlalchemy.engine.Engine SELECT zaposleni.id AS zaposleni_id, zaposleni.ime AS zaposleni_ime, zaposleni.datum_zaposlitve AS zaposleni_datum_zaposlitve, zaposleni.oddelek_id AS zaposleni_oddelek_id 
FROM zaposleni 
WHERE ? = zaposleni.oddelek_id
2025-06-05 10:36:55,415 INFO sqlalchemy.engine.Engine [generated in 0.00048s] (1,)


(Oddelek[1, IT, [Zaposleni[1, Janez, 2025-06-05 08:35:53, 1]]],
 Zaposleni[1, Janez, 2025-06-05 08:35:53, 1])

In [9]:
(zaposleni2 := Zaposleni(ime="Micka"))

Zaposleni[None, Micka, None, None]

In [10]:
session.add(zaposleni2)
session.commit()
zaposleni2

2025-06-05 10:40:52,729 INFO sqlalchemy.engine.Engine INSERT INTO zaposleni (ime, datum_zaposlitve, oddelek_id) VALUES (?, CURRENT_TIMESTAMP, ?)
2025-06-05 10:40:52,730 INFO sqlalchemy.engine.Engine [cached since 299s ago] ('Micka', None)
2025-06-05 10:40:52,731 INFO sqlalchemy.engine.Engine COMMIT
2025-06-05 10:40:52,743 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-06-05 10:40:52,744 INFO sqlalchemy.engine.Engine SELECT zaposleni.id AS zaposleni_id, zaposleni.ime AS zaposleni_ime, zaposleni.datum_zaposlitve AS zaposleni_datum_zaposlitve, zaposleni.oddelek_id AS zaposleni_oddelek_id 
FROM zaposleni 
WHERE zaposleni.id = ?
2025-06-05 10:40:52,744 INFO sqlalchemy.engine.Engine [generated in 0.00060s] (2,)


Zaposleni[2, Micka, 2025-06-05 08:40:52, None]

In [11]:
zaposleni2.oddelek = oddelek1
session.commit()
oddelek1

2025-06-05 10:41:50,837 INFO sqlalchemy.engine.Engine SELECT oddelek.id AS oddelek_id, oddelek.naziv AS oddelek_naziv 
FROM oddelek 
WHERE oddelek.id = ?
2025-06-05 10:41:50,839 INFO sqlalchemy.engine.Engine [generated in 0.00181s] (1,)
2025-06-05 10:41:50,845 INFO sqlalchemy.engine.Engine UPDATE zaposleni SET oddelek_id=? WHERE zaposleni.id = ?
2025-06-05 10:41:50,846 INFO sqlalchemy.engine.Engine [generated in 0.00127s] (1, 2)
2025-06-05 10:41:50,848 INFO sqlalchemy.engine.Engine COMMIT
2025-06-05 10:41:50,860 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-06-05 10:41:50,860 INFO sqlalchemy.engine.Engine SELECT oddelek.id AS oddelek_id, oddelek.naziv AS oddelek_naziv 
FROM oddelek 
WHERE oddelek.id = ?
2025-06-05 10:41:50,861 INFO sqlalchemy.engine.Engine [cached since 295.4s ago] (1,)
2025-06-05 10:41:50,862 INFO sqlalchemy.engine.Engine SELECT zaposleni.id AS zaposleni_id, zaposleni.ime AS zaposleni_ime, zaposleni.datum_zaposlitve AS zaposleni_datum_zaposlitve, zaposleni.oddel

Oddelek[1, IT, [Zaposleni[1, Janez, 2025-06-05 08:35:53, 1], Zaposleni[2, Micka, 2025-06-05 08:40:52, 1]]]

In [12]:
zaposleni1

Zaposleni[1, Janez, 2025-06-05 08:35:53, 1]

Izpišimo trenutno vsebino baze.

In [13]:
session.query(Zaposleni).all()

2025-06-05 10:42:38,961 INFO sqlalchemy.engine.Engine SELECT zaposleni.id AS zaposleni_id, zaposleni.ime AS zaposleni_ime, zaposleni.datum_zaposlitve AS zaposleni_datum_zaposlitve, zaposleni.oddelek_id AS zaposleni_oddelek_id 
FROM zaposleni
2025-06-05 10:42:38,963 INFO sqlalchemy.engine.Engine [generated in 0.00116s] ()


[Zaposleni[1, Janez, 2025-06-05 08:35:53, 1],
 Zaposleni[2, Micka, 2025-06-05 08:40:52, 1]]

In [14]:
session.query(Oddelek).all()

2025-06-05 10:43:01,329 INFO sqlalchemy.engine.Engine SELECT oddelek.id AS oddelek_id, oddelek.naziv AS oddelek_naziv 
FROM oddelek
2025-06-05 10:43:01,331 INFO sqlalchemy.engine.Engine [generated in 0.00166s] ()


[Oddelek[1, IT, [Zaposleni[1, Janez, 2025-06-05 08:35:53, 1], Zaposleni[2, Micka, 2025-06-05 08:40:52, 1]]]]

Izbrišimo `oddelek1`.

In [15]:
session.delete(oddelek1)
session.commit()

2025-06-05 10:43:26,176 INFO sqlalchemy.engine.Engine DELETE FROM zaposleni WHERE zaposleni.id = ?
2025-06-05 10:43:26,178 INFO sqlalchemy.engine.Engine [generated in 0.00188s] ((1,), (2,))
2025-06-05 10:43:26,179 INFO sqlalchemy.engine.Engine DELETE FROM oddelek WHERE oddelek.id = ?
2025-06-05 10:43:26,180 INFO sqlalchemy.engine.Engine [generated in 0.00063s] (1,)
2025-06-05 10:43:26,181 INFO sqlalchemy.engine.Engine COMMIT


Spet izpišimo trenutno vsebino baze.

In [16]:
session.query(Zaposleni).all()

2025-06-05 10:44:07,218 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-06-05 10:44:07,220 INFO sqlalchemy.engine.Engine SELECT zaposleni.id AS zaposleni_id, zaposleni.ime AS zaposleni_ime, zaposleni.datum_zaposlitve AS zaposleni_datum_zaposlitve, zaposleni.oddelek_id AS zaposleni_oddelek_id 
FROM zaposleni
2025-06-05 10:44:07,222 INFO sqlalchemy.engine.Engine [cached since 88.26s ago] ()


[]

In [17]:
session.query(Oddelek).all()

2025-06-05 10:44:12,444 INFO sqlalchemy.engine.Engine SELECT oddelek.id AS oddelek_id, oddelek.naziv AS oddelek_naziv 
FROM oddelek
2025-06-05 10:44:12,447 INFO sqlalchemy.engine.Engine [cached since 71.12s ago] ()


[]

Ustvarimo naslednjega zaposlenega in izpišimo datum zaposlitve.

In [18]:
zaposleni2 = Zaposleni(ime="Francka")
session.add(zaposleni2)
print(zaposleni2.datum_zaposlitve)

None


Privzeta vrednost se nastavi šele, ko kličemo `commit`.

In [19]:
session.commit()
print(zaposleni2.datum_zaposlitve)

2025-06-05 10:44:34,952 INFO sqlalchemy.engine.Engine INSERT INTO zaposleni (ime, datum_zaposlitve, oddelek_id) VALUES (?, CURRENT_TIMESTAMP, ?)
2025-06-05 10:44:34,955 INFO sqlalchemy.engine.Engine [cached since 521.2s ago] ('Francka', None)
2025-06-05 10:44:34,958 INFO sqlalchemy.engine.Engine COMMIT
2025-06-05 10:44:34,967 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-06-05 10:44:34,968 INFO sqlalchemy.engine.Engine SELECT zaposleni.id AS zaposleni_id, zaposleni.ime AS zaposleni_ime, zaposleni.datum_zaposlitve AS zaposleni_datum_zaposlitve, zaposleni.oddelek_id AS zaposleni_oddelek_id 
FROM zaposleni 
WHERE zaposleni.id = ?
2025-06-05 10:44:34,969 INFO sqlalchemy.engine.Engine [cached since 222.2s ago] (1,)
2025-06-05 08:44:34


Za privzeto vrednost smo uporabili `func.now()`. Izpišimo njeno vrednost pred in po interakciji z bazo.

In [20]:
func.now()

<sqlalchemy.sql.functions.now at 0x77365bb9c050; now>

In [21]:
rs = session.execute(select(func.now()))
rs.fetchone()

2025-06-05 10:45:31,838 INFO sqlalchemy.engine.Engine SELECT CURRENT_TIMESTAMP AS now_1
2025-06-05 10:45:31,840 INFO sqlalchemy.engine.Engine [generated in 0.00177s] ()


(datetime.datetime(2025, 6, 5, 8, 45, 31),)

Pobrišimo vse oddelke in uporabnike v njih.

In [22]:
for oddelek in session.query(Oddelek).all():
    session.delete(oddelek)
session.commit()

2025-06-05 10:46:00,347 INFO sqlalchemy.engine.Engine SELECT oddelek.id AS oddelek_id, oddelek.naziv AS oddelek_naziv 
FROM oddelek
2025-06-05 10:46:00,348 INFO sqlalchemy.engine.Engine [cached since 179s ago] ()
2025-06-05 10:46:00,349 INFO sqlalchemy.engine.Engine COMMIT


Še enkrat preglejmo zaposlene.

In [23]:
session.query(Zaposleni).all()

2025-06-05 10:46:19,763 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-06-05 10:46:19,767 INFO sqlalchemy.engine.Engine SELECT zaposleni.id AS zaposleni_id, zaposleni.ime AS zaposleni_ime, zaposleni.datum_zaposlitve AS zaposleni_datum_zaposlitve, zaposleni.oddelek_id AS zaposleni_oddelek_id 
FROM zaposleni
2025-06-05 10:46:19,769 INFO sqlalchemy.engine.Engine [cached since 220.8s ago] ()


[Zaposleni[1, Francka, 2025-06-05 08:44:34, None]]

Ustvarimo dva oddelka in tri zaposlene, ki jih vstavimo v oddelke.

In [24]:
IT = Oddelek(naziv="IT")
finance = Oddelek(naziv="Finance")
janez = Zaposleni(ime="Janez", oddelek=IT)
metka = Zaposleni(ime="Metka", oddelek=finance)
session.add(IT)
session.add(finance)
session.add(janez)
session.add(metka)
session.commit()

2025-06-05 10:46:38,875 INFO sqlalchemy.engine.Engine INSERT INTO oddelek (naziv) VALUES (?)
2025-06-05 10:46:38,876 INFO sqlalchemy.engine.Engine [cached since 645.1s ago] ('IT',)
2025-06-05 10:46:38,877 INFO sqlalchemy.engine.Engine INSERT INTO oddelek (naziv) VALUES (?)
2025-06-05 10:46:38,878 INFO sqlalchemy.engine.Engine [cached since 645.1s ago] ('Finance',)
2025-06-05 10:46:38,879 INFO sqlalchemy.engine.Engine INSERT INTO zaposleni (ime, datum_zaposlitve, oddelek_id) VALUES (?, CURRENT_TIMESTAMP, ?)
2025-06-05 10:46:38,879 INFO sqlalchemy.engine.Engine [cached since 645.1s ago] ('Janez', 1)
2025-06-05 10:46:38,880 INFO sqlalchemy.engine.Engine INSERT INTO zaposleni (ime, datum_zaposlitve, oddelek_id) VALUES (?, CURRENT_TIMESTAMP, ?)
2025-06-05 10:46:38,881 INFO sqlalchemy.engine.Engine [cached since 645.1s ago] ('Metka', 2)
2025-06-05 10:46:38,881 INFO sqlalchemy.engine.Engine COMMIT


In [25]:
katka = Zaposleni(ime="Katka", oddelek=finance)
session.add(katka)
session.commit()

2025-06-05 10:46:46,790 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-06-05 10:46:46,793 INFO sqlalchemy.engine.Engine SELECT oddelek.id AS oddelek_id, oddelek.naziv AS oddelek_naziv 
FROM oddelek 
WHERE oddelek.id = ?
2025-06-05 10:46:46,795 INFO sqlalchemy.engine.Engine [cached since 296s ago] (2,)
2025-06-05 10:46:46,798 INFO sqlalchemy.engine.Engine INSERT INTO zaposleni (ime, datum_zaposlitve, oddelek_id) VALUES (?, CURRENT_TIMESTAMP, ?)
2025-06-05 10:46:46,799 INFO sqlalchemy.engine.Engine [cached since 653.1s ago] ('Katka', 2)
2025-06-05 10:46:46,800 INFO sqlalchemy.engine.Engine COMMIT


/tmp/ipykernel_118042/4238068839.py:1: RemovedIn20Warning: Deprecated API features detected! These feature(s) are not compatible with SQLAlchemy 2.0. To prevent incompatible upgrades prior to updating applications, ensure requirements files are pinned to "sqlalchemy<2.0". Set environment variable SQLALCHEMY_WARN_20=1 to show all deprecation warnings.  Set environment variable SQLALCHEMY_SILENCE_UBER_WARNING=1 to silence this message. (Background on SQLAlchemy 2.0 at: https://sqlalche.me/e/b8d9)
  katka = Zaposleni(ime="Katka", oddelek=finance)


Preštejmo zaposlene.

In [26]:
session.query(Zaposleni).count()

2025-06-05 10:47:37,810 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-06-05 10:47:37,814 INFO sqlalchemy.engine.Engine SELECT count(*) AS count_1 
FROM (SELECT zaposleni.id AS zaposleni_id, zaposleni.ime AS zaposleni_ime, zaposleni.datum_zaposlitve AS zaposleni_datum_zaposlitve, zaposleni.oddelek_id AS zaposleni_oddelek_id 
FROM zaposleni) AS anon_1
2025-06-05 10:47:37,814 INFO sqlalchemy.engine.Engine [generated in 0.00061s] ()


4

Izpišimo ime enega zaposlenega, ki se začne na črko K.

In [27]:
session.query(Zaposleni).filter(Zaposleni.ime.startswith("K")).one().ime

2025-06-05 10:48:05,317 INFO sqlalchemy.engine.Engine SELECT zaposleni.id AS zaposleni_id, zaposleni.ime AS zaposleni_ime, zaposleni.datum_zaposlitve AS zaposleni_datum_zaposlitve, zaposleni.oddelek_id AS zaposleni_oddelek_id 
FROM zaposleni 
WHERE (zaposleni.ime LIKE ? || '%')
2025-06-05 10:48:05,318 INFO sqlalchemy.engine.Engine [generated in 0.00193s] ('K',)


'Katka'

Izpišimo ime enega zaposlenega, ki je hkrati v oddelku Finance in se njegovo ime začne na črko K.

In [28]:
session.query(Zaposleni).join(Zaposleni.oddelek).filter(
    Zaposleni.ime.startswith('K'), Oddelek.naziv == 'Finance').all()

2025-06-05 10:49:08,895 INFO sqlalchemy.engine.Engine SELECT zaposleni.id AS zaposleni_id, zaposleni.ime AS zaposleni_ime, zaposleni.datum_zaposlitve AS zaposleni_datum_zaposlitve, zaposleni.oddelek_id AS zaposleni_oddelek_id 
FROM zaposleni JOIN oddelek ON oddelek.id = zaposleni.oddelek_id 
WHERE (zaposleni.ime LIKE ? || '%') AND oddelek.naziv = ?
2025-06-05 10:49:08,897 INFO sqlalchemy.engine.Engine [generated in 0.00241s] ('K', 'Finance')


[Zaposleni[4, Katka, 2025-06-05 08:46:46, 2]]

Izpišimo število zaposlenih, ki so bili zaposleni v preteklosti.

In [29]:
session.query(Zaposleni).filter(Zaposleni.datum_zaposlitve < func.now()).count()

2025-06-05 10:49:30,568 INFO sqlalchemy.engine.Engine SELECT count(*) AS count_1 
FROM (SELECT zaposleni.id AS zaposleni_id, zaposleni.ime AS zaposleni_ime, zaposleni.datum_zaposlitve AS zaposleni_datum_zaposlitve, zaposleni.oddelek_id AS zaposleni_oddelek_id 
FROM zaposleni 
WHERE zaposleni.datum_zaposlitve < CURRENT_TIMESTAMP) AS anon_1
2025-06-05 10:49:30,569 INFO sqlalchemy.engine.Engine [generated in 0.00102s] ()


4